Hola &#x1F600;

Soy **Hesus Garcia**  como "Jesús" pero con H. Sé que puede ser confuso al principio, pero una vez que lo recuerdes, ¡nunca lo olvidarás! &#x1F31D;	. Como revisor de código de Triple-Ten, estoy emocionado de examinar tus proyectos y ayudarte a mejorar tus habilidades en programación. si has cometido algún error, no te preocupes, pues ¡estoy aquí para ayudarte a corregirlo y hacer que tu código brille! &#x1F31F;. Si encuentro algún detalle en tu código, te lo señalaré para que lo corrijas, ya que mi objetivo es ayudarte a prepararte para un ambiente de trabajo real, donde el líder de tu equipo actuaría de la misma manera. Si no puedes solucionar el problema, te proporcionaré más información en la próxima oportunidad. Cuando encuentres un comentario,  **por favor, no los muevas, no los modifiques ni los borres**. 

Revisaré cuidadosamente todas las implementaciones que has realizado para cumplir con los requisitos y te proporcionaré mis comentarios de la siguiente manera:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>
</div>

</br>

**¡Empecemos!**  &#x1F680;

# Tabla de Contenidos

* [Introducción](#intro)
* [1. Información general](#info)
    * [1.1 Preparar los datos](#preparar)
* [2. Análisis Exploratorio de datos](#analisis)  
* [3. Análisis de la prueba A/B](#resultado)
* [4. Conclusiones](#conclusiones)

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Me parece genial que hayas incluido una tabla de contenidos en tu documento, esto facilitará la navegación y comprensión de los temas que estás tratando. ¡Sigue así!</div>
</div>


<a id='intro'></a>
# Introducción

<a id='info'></a>
## Información general

Analizaremos una tienda en línea internacional donde se realizo una prueba A/B, los resultados de las pruebas están disponibles y las especificaciones técnicas, la interpretación de estos datos son importantes para comprender el impacto de los cambios introducidos en el sistema de recomendaciones.

**Descripción técnica**

- Nombre de la prueba: `recommender_system_test`
- Grupos: А (control), B (nuevo embudo de pago)
- Launch date: 2020-12-07
- Fecha en la que dejaron de aceptar nuevos usuarios: 2020-12-21
- Fecha de finalización: 2021-01-01
- Audiencia: 15% de los nuevos usuarios de la región de la UE
- Propósito de la prueba: probar cambios relacionados con la introducción de un sistema de recomendaciones mejorado
- Resultado esperado: dentro de los 14 días posteriores a la inscripción, los usuarios mostrarán una mejor conversión en vistas de la página del producto (el evento `product_page`), instancias de agregar artículos al carrito de compras (`product_card`) y compras (`purchase`). En cada etapa del embudo `product_page → product_card → purchase`, habrá al menos un 10% de aumento.
- Número previsto de participantes de la prueba: 6 000

Tenemos cuatro datasets para desarrollar el proyecto que contiene la siguiente información:

`ab_project_marketing_events_us.csv`: Calendario de eventos de marketing para 2020

- `name`: el nombre del evento de marketing
- `regions`: regiones donde se llevará a cabo la campaña publicitaria
- `start_dt`: fecha de inicio de la campaña
- `finish_dt`: fecha de finalización de la campaña

`final_ab_new_users_upd_us.csv`: Todos los usuarios que se registraron en la tienda en línea desde el 7 hasta el 21 de diciembre de 2020

- `user_id`
- `first_date`: fecha de inscripción
- `region`
- `device`: dispositivo utilizado para la inscripción

`final_ab_events_upd_us.csv`: Todos los eventos de los nuevos usuarios en el período comprendido entre el 7 de diciembre de 2020 y el 1 de enero de 2021

- `user_id`
- `event_dt`: fecha y hora del evento
- `event_name`: nombre del tipo de evento
- `details`: datos adicionales sobre el evento (por ejemplo, el pedido total en USD) para los eventos purchase

`final_ab_participants_upd_us.csv`: Tabla con los datos de los participantes de la prueba

- `user_id`
- `ab_test`: nombre de la prueba
- `group`: el grupo de prueba al que pertenecía el usuario

<a id='preparar'></a>
### Preparar los datos

Importemos las librerias que vamos a utilizar durante el desarrollo del análisis

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import math as mth
import scipy.stats as stats
from plotly.subplots import make_subplots
from datetime import timedelta

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Quería proporcionarte algunos comentarios sobre la organización de los imports en tu código.     
Es preferible agrupar los imports siguiendo el siguiente orden:

Imports de la biblioteca estándar de Python.<br>
Imports de bibliotecas de terceros relacionadas.<br>
Imports específicos de la aplicación local o biblioteca personalizada.<br>
Para mejorar la legibilidad del código, también es recomendable dejar una línea en blanco entre cada grupo de imports, pero solo un import por línea.<br>
Te dejo esta referencia con ejemplos:  
https://pep8.org/#imports

</div>

In [4]:
## cargue de los datasets

marketing = pd.read_csv("ab_project_marketing_events_us.csv")
participants = pd.read_csv("final_ab_participants_upd_us.csv")
inscriptions = pd.read_csv("final_ab_new_users_upd_us.csv")
events = pd.read_csv("final_ab_events_upd_us.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'ab_project_marketing_events_us.csv'

Revisar cada dataset para conocer la información con la que trabajaremos y este correcta

**Calendario de eventos de marketing para 2020**

In [ ]:
#Revisar información mktn
marketing.info()

In [ ]:
##Verificar valores ausentes
marketing.isna().sum()

In [ ]:
##Verificar valores duplicados
marketing.duplicated().sum()

A lo largo de año se realizaron 14 campañas de marketing, no hay valores ausentes, ni duplicados en la data.
El tipo de objetos para las columnas `start_dt` y `finish_dt`esta en objetc, es necesario cambiarlo a fecha

In [ ]:
##Corregir tipo de datos
marketing['start_dt'] = pd.to_datetime(marketing['start_dt'], format='%Y-%m-%d')
marketing['finish_dt'] = pd.to_datetime(marketing['finish_dt'], format='%Y-%m-%d')

In [ ]:
marketing.info()

In [ ]:
##Ordenar los datos de menor a mayor con respecto a la fecha de inicio
marketing.sort_values('start_dt')

Con respecto a las campañas de Marketing, la campaña Christmas&New Year Promo, podría estar influyendo en la prueba, en la parte final, es decir en los últimos 6 dias que dura el experimiento de 24 días, además esa campaña afecta la unión europea foco de estudio.

**Eventos de los nuevos usuarios en el período comprendido entre el 7 de diciembre de 2020 y el 1 de enero de 2021**

In [ ]:
events.info()

In [ ]:
#Revisar valores ausentes
events.isna().sum()

In [ ]:
#Revisar valores duplicados
events.duplicated().sum()

In [ ]:
#Revisar valores unicos
events['event_name'].unique()

In [ ]:
events[events['event_name'] == 'login' ].head()

<div class="alert alert-block alert-success">
    <b>Comentarios del Revisor</b> <a class="tocSkip"></a><br>
Correcto, info(), head()  son herramientas esceneciales que nos ayudaran a hacer un análisis exploratorio inicial. Opcionalmente podrías siempre incluir describe() para tener mejor idea de los valores que toman tus varibales. POr otra parte, buen trabajo por convertir los tipos de datos de fecha al formato adecuado. Continúa con el buen trabajo! </div>

In [ ]:
missing_details = events[events['event_name'] == 'purchase']['details'].isna().any()
missing_details

No se tienes valores duplicados y con respecto a los valores ausentes que aparecen en la columna details, hacen referencia a las tareas de 'product_cart', 'product_page', 'login' en las cuales no se realiza ningún pago es por este motivo el valor ausente, verificamos y con respecto a purchase no hay ningún valor ausente

In [ ]:
events["event_name"].value_counts()

**Datos participantes de la prueba**

In [ ]:
#Revisar dataset
participants.head()

In [ ]:
# Verificar que esten los dos grupos A y B
participants['group'].unique()

In [ ]:
## Verificar duplicados de usuarios
group_a = participants.query('group == "A"')
group_b = participants.query('group == "B"')

match = participants["user_id"].isin(group_a["user_id"]) & participants["user_id"].isin(group_b["user_id"])

duplicates = participants[match]

duplicates

<div class="alert alert-block alert-success">
    <b>Comentarios del Revisor</b> <a class="tocSkip"></a><br>
Correcto se dividen los grupos de participantes y se busca que sean independientes al eliminar duplicados. </div>

In [ ]:
#Filtro verificar usuario duplicado
participants[participants['user_id'] == 'FE2AF0E94DBD470E']

Encontramos 882 valores duplicados con respecto a los usuarios, estos usuarios estan dentro de la prueba recommender_system_test e interface_eu_test, para este caso y como usaremos una prueba Z y los resultados pueden variar, eliminaremos los valores duplicados y trabajaremos solo con los usuarios que entraron directamente a la pueba recommender_system_test.

In [ ]:
#Eliminar valores de usuario duplicados
filtered_participants = participants[~match]

In [ ]:
filtered_participants.head()

In [ ]:
# Verificar que no tengamos otros tipos de prueba
filtered_participants['ab_test'].unique()

Observamos que tenemos otro tipo de prueba interface_eu_test, asi que filtraremos los datos solo para que no aparezca la de nuestro objeto de interes recommender_system_test

In [ ]:
participants_test = filtered_participants.query('ab_test == "recommender_system_test"').reset_index(drop=True)

In [ ]:
#Verificar que los datos esten correctos
participants_test['ab_test'].unique()

In [ ]:
participants_test.info()

In [ ]:
#Verificar valores ausentes
participants_test.isna().sum()

In [ ]:
#Verificar valores duplicados
participants_test.duplicated().sum()

Contamos con 3234 datos, no se presentan valores ausentes, ni duplicados.

**Datos de los usuarios que se registraron en la tienda en línea desde el 7 hasta el 21 de diciembre de 2020**

In [ ]:
inscriptions.info()

In [ ]:
#Verificar valores ausentes
inscriptions.isna().sum()

In [ ]:
#Verificar valores duplicados
inscriptions.duplicated().sum()

No se presentan valores ausentes ni duplicados.
Se debe convertir la columna `first_date` al tipo de dato fecha, lo realizaremos más adelante.

Uniremos el dataset de participantes con el de incripciones

In [ ]:
#Unión datasets
participants_inscriptions = pd.merge(inscriptions, participants_test,
    how="inner",
    on="user_id"
)

In [ ]:
participants_inscriptions.head()

En el estudio estamos interesados en la región de la unión europea, revisemos si tenemos otras regiones presentes.

In [ ]:
#Verificar regiones en el dataset
participants_inscriptions['region'].value_counts()

Tenemos N.America, APAC y CIS, lo que haremos es eliminarlas y quedarnos con solo EU.

In [ ]:
#Filtro aplicado para solo los usuarios de la región EU
participants_test_eu = participants_inscriptions.query('region == "EU"').reset_index(drop=True)

Teniendo esta información la uniremos con la información de eventos para que nos quede la data con los usuarios que hacen parte del experimento

In [ ]:
#Unión de datasets
test_ab = pd.merge(
    events,
    participants_test_eu,
    how="inner",
    on="user_id"
)

In [ ]:
test_ab.head()

In [ ]:
test_ab.info()

corregiremos los tipos de datos para `event_dt` y `first_date` a tipo  fecha

In [ ]:
#cambio tipo de dato objeto a fecha
test_ab['first_date'] = pd.to_datetime(test_ab['first_date'])
test_ab['event_dt'] = pd.to_datetime(test_ab['event_dt'])

In [ ]:
test_ab.info()

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
¡Muy bien! 👏👏 Los cálculos de esta sección están correctos y eso es un gran logro.</div>

<a id='analisis'></a>
## Análisis Exploratorio de datos

In [ ]:
#Usuarios unicos
test_ab['user_id'].nunique()

La cantidad de usuarios unicos que tenemos es de 3040

In [ ]:
#Cantidad de eventos por usuarios
test_ab.groupby('user_id')['event_name'].count().mean().round(2)

En promedio cada usuario tiene alrededor de 6 eventos registrados

In [5]:
#Econtrar valor mínimo y máximo de fechas

print(f'Valor Mínimo:', test_ab['first_date'].min())

print(f'Valor Máximo:', test_ab['first_date'].max())

NameError: name 'test_ab' is not defined

Vemos que la fecha de incricipción esta dentro del rango del estudio

In [ ]:
#Econtrar valor mínimo y máximo de fechas

print(f'Valor Mínimo:', test_ab['event_dt'].min())

print(f'Valor Máximo:', test_ab['event_dt'].max())

Con respecto a los eventos que cada usuario realizo estan entre el 7 y 30 de diciembre, no se tiene información del 31 de diciembre y 1 de enero del 2021 que duraba el estudio

In [ ]:
test_ab.head()

In [ ]:
#Gráfico de usuarios únicos por grupo y evento
events_by_group = test_ab.groupby(['group', 'event_name']).agg({"user_id": ["count", "nunique"]}).unstack(fill_value=0)
events_by_group = events_by_group.reset_index()
events_by_group['group'] = events_by_group['group'].astype(str)

plot_data = (
    events_by_group
    .melt(id_vars=[('group', '', '')], value_vars=[c for c in events_by_group.columns if "nunique" in c])
    .rename(columns={"variable_2": "event", ('group', '', ''): "group"})
)

plot_data.sort_values(by='value', inplace=True, ascending=False)
color_map = {'246': 'gold', '247': 'coral', '248': 'steelblue'}

fig = px.bar(plot_data,
             x='event', y='value', color="group",
             labels={'event': 'Tipo de evento', 'value': 'Frecuencia'},
             title='Usuarios únicos por grupo y evento',
             barmode='group',
             color_discrete_map=color_map,
             height=450, width=950
            )

fig.show()

El número de usuarios únicos por grupo y evento es muy diferente en ambas muestras, teniendo más data la muestra A que la muestra B.

In [ ]:
#Gráfico histograma registros a través del tiempo
fig = px.histogram(test_ab, x='event_dt', nbins=15, title='Registros a través del tiempo')

fig.show()

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
¡Muy bien! Buen trabajo con la generación de gráficos has generado gráficos adecuados por grupo. Destaco que has generado las etiquetas adecuadas de los ejes y personalizado los colores.</div>

El día con mayor cantidad de eventos registrados en la plataforma fue el 21 de diciembre, alcanzando un total de 3613 eventos. Observamos que no hay una tendencia de incremento notable con la implementación de la campaña de marketing Christmas&New Year Promo que comenzó el 25 de diciembre. De hecho, durante esos días, el uso de la plataforma fue menor, especialmente a partir del 24 de diciembre. Es interesante destacar que los días previos a la Navidad registraron un mayor volumen de ventas en comparación con los días posteriores.

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
En esta sección te aconsejaría realizar el cálculo porcentual de la diferente entre las dos campañas para ser más específico al describir incrementos. </div>

In [ ]:
#Filtro para el grupo A yB
group_a = test_ab.query("group == 'A'")
group_b = test_ab.query("group == 'B'")

In [ ]:
# Definir el rango de 14 días después de la inscripción
start_date = test_ab['first_date'].min()
end_date = start_date + timedelta(days=14)

# Filtrar eventos que ocurrieron dentro de los 14 días
group_a_14days = group_a[group_a['event_dt'] <= end_date]
group_b_14days = group_b[group_b['event_dt'] <= end_date]

In [ ]:
# Creación del Embudo de Conversión para el Grupo A 
funnel_a = group_a_14days.pivot_table(
    index="event_name",
    values="user_id",
    aggfunc="nunique"
)

# Cálculo de las Tasas de Conversión para el Grupo A 
funnel_a["proportion"] = funnel_a["user_id"] / len(group_a["user_id"].unique())

# Ordenamiento del Embudo por Número de Usuarios Únicos
funnel_a = funnel_a.sort_values(by='user_id', ascending=False).reset_index()
funnel_a

In [ ]:
# Creación del Embudo de Conversión para el GrupoB 
funnel_b = group_b_14days.pivot_table(
    index="event_name",
    values="user_id",
    aggfunc="nunique"
)

# Cálculo de las Tasas de Conversión para el Grupo B
funnel_b["proportion"] = funnel_b["user_id"] / len(group_b["user_id"].unique())

# Ordenamiento del Embudo por Número de Usuarios Únicos
funnel_b = funnel_b.sort_values(by='user_id', ascending=False).reset_index()
funnel_b

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Gran manejo de la función pivot table 💪💻</div>

In [ ]:
# Crear subgráficos con 1 fila y 2 columnas
fig = make_subplots(rows=1, cols=2, subplot_titles=['Embudo Muestra A', 'Embudo Muestra B'])

# Añadir el embudo para la muestra A
fig.add_trace(
    go.Funnel(
        y=funnel_a['event_name'],
        x=funnel_a['user_id'],
        textposition="inside",
        textinfo="value+percent initial",
        marker=dict(color='powderblue'),
        connector=dict(line=dict(color="powderblue", width=4)),
    ),
    row=1, col=1
)

# Añadir el embudo para la muestra B
fig.add_trace(
    go.Funnel(
        y=funnel_b['event_name'],
        x=funnel_b['user_id'],
        textposition="inside",
        textinfo="value+percent initial",
        marker=dict(color='mediumslateblue'),
        connector=dict(line=dict(color="mediumslateblue", width=4)),
    ),
    row=1, col=2
)

fig.update_layout(
    title_text="Embudo de Usuarios por Evento",
    yaxis_title="Eventos",
    showlegend=False  
)

fig.show()


Podemos observar que no se cumple el criterio de aumento del 10% en cada etapa del embudo dentro de los 14 días posteriores a la inscripción.

`Muestra A`:

Mayor porcentaje de usuarios que visitan la página del producto: 64% en comparación con el 57% de la muestra B. Esto podría indicar un mayor interés en el producto por parte de los usuarios de la muestra A.
Menor porcentaje de usuarios que completan la compra: 32% en comparación con el 28% de la muestra B. A pesar del mayor interés inicial, la muestra A tiene una tasa de conversión menor.
Caída significativa entre la página del producto y el carrito de compra: 31% de los usuarios que visitan la página del producto no agregan el producto al carrito. Esto podría indicar problemas con la experiencia de compra o con el precio del producto.

`Muestra B`:

Menor porcentaje de usuarios que visitan la página del producto: 57% en comparación con el 64% de la muestra A. Esto podría indicar un menor interés en el producto por parte de los usuarios de la muestra B.
Similar porcentaje de usuarios que completan la compra: 28% en comparación con el 32% de la muestra A. La muestra B tiene una tasa de conversión similar a la muestra A, a pesar de tener un menor interés inicial.
Caída significativa entre el login y la página del producto: 43% de los usuarios que inician sesión no visitan la página del producto. Esto podría indicar problemas con la experiencia de usuario después del inicio de sesión o con la visibilidad del producto.


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
¡Felicitaciones por incluir conclusiones intermedias en su proyecto! Esto es una práctica excelente ya que ayuda a comprender mejor los datos y a tomar decisiones informadas en la siguiente etapa del análisis. Continúa haciendo esto en futuros proyectos.</div>

<a id='resultado'></a>
## Análisis de la prueba A/B 

`H0 - Hipótesis Nula`: La proporción de eventos en el grupo A es igual a la proporción de eventos en el grupo B.

`H1 - Hipótesis Alternativa`: La proporción de eventos en el grupo A es diferente a la proporción de eventos en el grupo B.

El alpha que utilizaremos es de 0.05

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Correcto. Se establecieron la hipótesis nula y alternativa y planteas el nivel de significancia adecuado. </div>

In [ ]:
# Cálculo del Número Total de Usuarios Únicos por Grupo
data_total = test_ab.groupby('group')['user_id'].nunique().reset_index()
data_total

In [ ]:
# Creación de una Tabla de Usuarios Únicos por Evento y Grupo
data_unique = test_ab.pivot_table(
    index='event_name', 
    values='user_id', 
    columns='group', aggfunc=lambda x: x.nunique()).sort_values(by='A', ascending=False).reset_index()
data_unique

In [ ]:
def hypothesis(group1, group2, event_name, alpha=0.05):
    
    # Filtrado por evento, grupo y usuarios únicos por evento
    successes1 = data_unique[data_unique['event_name'] == event_name][group1].iloc[0]
    successes2 = data_unique[data_unique['event_name'] == event_name][group2].iloc[0]
    
    # Usuarios totales únicos
    trials1 = data_total[data_total['group'] == group1]['user_id'].iloc[0]
    trials2 = data_total[data_total['group'] == group2]['user_id'].iloc[0]
    
    # Proporción grupo 1
    p1 = successes1 / trials1
    
    # Proporción grupo 2
    p2 = successes2 / trials2

    # Proporción grupos combinados
    p_combined = (successes1 + successes2) / (trials1 + trials2)
    difference = p1 - p2   
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials1 + 1/trials2))
    distr = stats.norm(0, 1) 
    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-value: ', p_value)

    if p_value < alpha:
        print("Rechazamos la Hipótesis Nula hay evidencia estadística significativa para afirmar que la proporción de eventos en el grupo A es diferente de la proporción de eventos en el grupo B para el evento", event_name, )
    else:
        print("No hay evidencia suficiente para afirmar que la proporción de eventos en el grupo A es diferente de la proporción de eventos en el grupo B para el evento ",event_name)


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Buen trabajo por plantear una función para hacer más reutilizable la función. Además usar el valor Z es lo más adecuado para este tipo de pruebas AB</div>

In [ ]:
#Pueba de hipotesis

for i in data_unique['event_name'].unique():
    hypothesis('A', 'B', i, alpha=0.05)
    print('\n')

En Conclusión, según los p-values obtenidos, hay evidencia estadística significativa para afirmar que la proporción de eventos es diferente entre los grupos para el evento 'product_page', pero no hay suficiente evidencia para afirmar diferencias significativas en los otros eventos.

<a id='conclusiones'></a>
## Conclusiones 

Se considera que el tiempo de la prueba fue muy corto para determinar resultados, no se evidencia que la campaña de marketing que se lanzo en los días finales del año haya tenido relevancia, antes las ventas disminuyeron despues de la fiesta de navidad y los datos recolectados estuvieron hasta el 30 de diciembre del 2020 no hasta el 1 de enero de 2021 que finalizaba la prueba.

El número previsto de partipante fue menor de los 6000 que se tenian previstos, el número de usuarios únicos fue de 3040.

el resultado esperado dentro de los 14 días posteriores a la inscripción, se esperaba un aumento del 10% pero se puede ver que comparado con el grupo A hubo una diminución.


Ambas muestras tienen una tasa de conversión relativamente baja, la muestra A tiene un mayor interés inicial en el producto, pero una menor tasa de conversión. Se debería analizar en detalle el proceso de compra para identificar los puntos de fricción que están impidiendo que los usuarios completen la compra.
La muestra B tiene un menor interés inicial en el producto, pero una tasa de conversión similar a la muestra A. Se debería analizar cómo se puede aumentar el interés en el producto y la visibilidad del mismo en el sitio web.

Para los eventos login, purchase, y product_cart, no hay evidencia suficiente para afirmar diferencias significativas en las proporciones de eventos entre los grupos A y B, la prueba indica evidencia estadística significativa para afirmar que la proporción de eventos en el grupo A es diferente del grupo B para el evento product_page. Esto puede indicar un impacto significativo del nuevo sistema de recomendaciones en la visualización de páginas de productos.

Se puede concluir que la prueba no cumplió con el objetivo planteado inicialmente de implementar un sistema de recomendaciones mejorado.

<div class="alert alert-block alert-success">    
<b>Comentario del revisor</b> <a class="tocSkip"></a>

¡Qué gran trabajo has hecho!  &#128077;  <b>Podemos aprobar el proyecto.</b><br>

<br>Quiero felicitarte por un trabajo excepcional y por la calidad de tu análisis.<br>
- Tienes un gran manejo del lenguaje python y usas la visualización de forma efectiva. Además aplicanste las pruebs AB de forma adecuado. Si te gustaría profundizar más en el tema te aconsejo repetir tus pruebas utilizando pruebas de permutación para comprobar las hipótesis.     

Te animo a que sigas aprendiendo y desafiando tu potencial en los próximos sprints. Estoy seguro de que tus habilidades y conocimientos serán valiosos en el futuro y te permitirán abordar problemas cada vez más complejos con éxito.
</div>